**Jonathan Rossi**  
*April 9, 2016*  
Python

---

# Perceptron Learning Algorithm for Classification #

This is an implementation of the Perceptron Learning Algorithm for classification, designed by Frank Rosenblatt (F. Rosenblatt, 'The Perceptron, a Perceiving and Recognizing Automaton,' Cornell Aeronautical Laboratory, 1957). As this is a learning experience for me, this implementation draws heavily on Sebastian Raschka's code in his book 'Python Machine Learning' (Packt Publishing, 2015). Thanks for a rock-solid explanation of this algorithm, Sebastian!

## Sections ##

* Design Overview
* Implementation
* Implementation Walkthrough

## Design Overview ##

In a nutshell, the algorithm finds the ideal weights (aka, coefficients) to apply to each feature in the data set, so that the dot product of the feature vector and the weights vector falls above or below a threshold, which predicts whether the observation (i.e., sample) is, or is not, of a particular class. The algorithm initializes the weights vector to the zero-vector, or a vector of small arbitrary numbers, then takes one observation at a time and, for each observation, computes the output value (i.e., the dot product of the weights vector and the feature vector), then updates the weights vector based on the difference between the ground truth value and the output value. For correct predictions, the weights vector is left unchanged. For incorrect predictions, individual values in the weights vector are adjusted to be more positive or more negative—in proportion to the magnitude of the corresponding feature vector—so as to "push" the prediction closer to the ground truth value.

## Implementation ##

In [5]:
import numpy as np

In [6]:
# Define the Perceptron model class.
class Perceptron(object):
    def __init__(self, learnrate = 0.01, n_iter = 10):
        # learnrate (float): the learning rate (between 0.0 and 0.1).
        # n_iter (int): the number of iterations over the training data.

        self.learnrate = learnrate
        self.n_iter = n_iter
        
    # Define fit function.
    def fit(self, X, y):
        # X (array-like, shape = [n_observations, n_features]): the training data in the form of vectors with
        #     n_observations observations and n_features features.
        # y (array-like, shape [n_observations]): the vector of output values (labels).

        # Initialize the weights vector and error-count list.
        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []
            # w_ (1d-array): the weights vector after fitting the current observation and updating the weights
            # errors_ (list): the number of misclassifications in the current epoch.

        # For each iteration, update the weights vector and register a misclassification if necessary.
        for _ in range(self.n_iter):
            errors = 0
                # errors (int): counter for misclassifications.
            for xi, target in zip(X,y):
                update = self.learnrate * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self

    # Define a "net input" function.
    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

    # Define a predict function.
    def predict(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, -1)

## Implementation Walkthrough ##

This is a detailed, step-by-step explanation of the Python implementation of the algorithm.

### Define the model class. ###

* **Define the initializer.**


* **Define a 'fit' function** that passes over each observation in the training data set, updates the weights vector each time, and tracks the number of misclassifications of observations.

    * Initialize the weights vector and error-count list (number of misclassifications per epoch (i.e., one full pass of the ML algorithm over the whole of the training data). By convention, use names that end in `_` for attributes of the Perceptron class that are not part of the initialization.
    
        * `zeros` creates an array of all zeros. `X.shape` returns a tuple of form `(n,m)` where `n` is the number of rows of `X` and `m` is the number of columns. So, `X.shape[1]` returns the second value of the tuple, namely the number of columns. We add `1` to that number in order to create a zeros vector with one additional value, which will represent `w_0`, the 1st coefficient (which will ultimately equal the negative of our threshold). We will also add a `1` to the beginning of the features vector. This essentially allows us to draw the class divider line at `0`, rather than at our threshold value, which helps simplify things. For a detailed explanation of the math behind this algorithm, see: https://en.wikipedia.org/wiki/Perceptron.
    
        * So, `np.zeros(1 + Z.shape[1])` returns an `1d-array` of all zeros of length `1 + X.shape[1]`. We also create an empty array `errors_` to track the number of misclassifications.
    
    * For each iteration, update the weights vector and register a misclassification if necessary.
        * For each observation, `xi`, and output value (predicted label), `target`, carry out the following steps. Note that `zip(X,y)` returns a tuple comprised of the first row (observation) in the array `X` and the corresponding output value in `y` (the ground truth value).
            * Define a new variable, `update`, and set it equal to the learning rate multiplied by the difference between `target`, the ground truth label value, and `self.predict(xi)`, the predicted label value for observation `xi`. Note, we define the `predict()` function below.
            * Add the updated weight amounts to each weight in the weights vector, aside from `w_0`.
            * Since, `w_0` is associated with the `1` that occupies the first position of the features vector, we don't multiply it by anything.
            * Track misclassification in `errors`, if necessary. If `update == 0.0`, the classification was correct. Otherwise, it was not. Note, `int(True)` returns `1` and `int(False)` returns `0`.
        * Update the list of misclassifications, `self.errors_`.
    
    
* **Define a 'net input' function.**
    
    * This is simply the dot product of the weights vector and the features vector. We will feed this into our "activation function" to see whether the observation is of the class in question, or is not. Note, the term "activation function" comes from the original Perceptron Model, which was meant to model a single neuron in the human brain. Given an input signal whose strength exceeds a particular threshold, the neuron will activate. If the signal is weaker than the threshold, the neuron will not fire. In our case, the neuron firing happens when the observation falls into the class, and the neuron does not fire when it does not. Note: `X` here is not to be confused with `X` (the `2d array`) in the `fit()` function.
    

* **Define a 'predict' function.**
    * This is the activation function referenced above. It returns either `1` if the observation is predicted to be of the class, and `-1` if the observation is predicted to not be of the class. Note: `X` here is not to be confused with `X` (the `2d array`) in the `fit()` function.
        * More specifically, `predict()` returns `1` if `net input()` returns a value greater than or equal to `0.0` (which we said is our "adjusted" threshold), and `-1` if `net input()` returns a value less than `0.0`. When the first argument of `np.where` is `True`, the second argument is returned and when the first argument is `False` the third argument is returned.